In [1]:
import datetime as dt
import numpy as np
import cvxpy
from alphamind.portfolio.linearbuilder import linear_builder

In [2]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    result = py_callable(n)
    elapsed = (dt.datetime.now() - start).total_seconds()
    return elapsed, result

def cvxpy_lp(n):
    w = cvxpy.Variable(n)

    bndl = np.zeros(n)
    bndu = 0.01 * np.ones(n)
    risk_constraints1 = np.ones((n,1))
    risk_constraints2 = np.zeros((n,1))
    risk_constraints2[0][0] = 1.
    risk_constraints2[1][0] = 1.
    risk_constraints = np.concatenate((risk_constraints1, risk_constraints2), axis=1)

    curr_risk_exposure = risk_constraints.T @ w
    risk_targets = np.array([1., 0.015])

    constraints = [w >= bndl,
                   w <= bndu,
                   curr_risk_exposure >= risk_targets,
                   curr_risk_exposure <= risk_targets]
    
    np.random.seed(1)
    er = np.random.randn(n)

    objective = cvxpy.Minimize(-w.T * er)
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return w, prob

In [3]:
print("{0:<8}{1:>12}{2:>12}{3:>12}{4:>12}{5:>12}{6:>15}".format('Scale(n)', 'time(ms)', 'feval', 'min(x)', 'max(x)', 'sum(x)', 'x(0) + x(1)'))

for n in range(200, 3200, 200):
    elapsed, result = time_function(cvxpy_lp, n)
    s = np.array(result[0].value).flatten()
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12f}{4:>12f}{5:>12f}{6:>15}".format(n, elapsed*1000, result[1].value, s.min(), s.max(), s.sum(), s[0] + s[1]))

Scale(n)    time(ms)       feval      min(x)      max(x)      sum(x)    x(0) + x(1)
200            18.00       -0.82   -0.000021    0.011996    1.0000100.013991955470964355
400            21.40       -1.28   -0.000000    0.010000    1.000000          0.015
600            21.00       -1.64   -0.000435    0.009980    1.0001740.014845697577983343
800            24.01       -1.83   -0.000802    0.010028    1.0002310.01474166785085744
1000           44.88       -1.89   -0.000166    0.010000    1.0001490.014851819526468822
1200           44.02       -2.07   -0.000571    0.009999    1.0001670.014833266833006432


d:\ProgramData\Anaconda3\lib\site-packages\cvxpy-1.0.8-py3.6-win-amd64.egg\cvxpy\problems\problem.py:614: RuntimeWarning: overflow encountered in long_scalars
  if self.max_big_small_squared < big*small**2:
d:\ProgramData\Anaconda3\lib\site-packages\cvxpy-1.0.8-py3.6-win-amd64.egg\cvxpy\problems\problem.py:615: RuntimeWarning: overflow encountered in long_scalars
  self.max_big_small_squared = big*small**2


1400           57.00       -2.31   -0.000504    0.009997    1.0002170.014785380519459417
1600           47.02       -2.90   -0.001650    0.009905    1.0004330.014667925787486116
1800           79.06       -2.67   -0.000885    0.009999    1.0002300.014771011591262167
2000           68.01       -2.81   -0.000333    0.010001    1.0002410.014758149340884413
2200           72.02       -3.71   -0.000849    0.009969    1.0004940.014536311105299406
2400           94.19       -2.46   -0.000536    0.010001    1.0000810.014917972574632015
2600          135.07       -2.54   -0.000105    0.010001    1.0000900.01490885499964943
2800          106.02       -3.40   -0.000983    0.010000    1.0002670.014733268551116162
3000          109.02       -3.77   -0.001612    0.010001    1.0003180.014680802417975643


In [4]:
def clp_lp(n):
    np.random.seed(1)
    er = np.random.randn(n)

    bndl = np.zeros(n)
    bndu = 0.01 * np.ones(n)
    risk_constraints1 = np.ones((n,1))
    risk_constraints2 = np.zeros((n,1))
    risk_constraints2[0][0] = 1.
    risk_constraints2[1][0] = 1.
    risk_constraints = np.concatenate((risk_constraints1, risk_constraints2), axis=1)
    risk_target = np.array([1., 0.015]), np.array([1., 0.015])
    
    result = linear_builder(er, bndl, bndu, risk_constraints, risk_target)
    return result

In [5]:
print("{0:<8}{1:>12}{2:>12}{3:>12}{4:>12}{5:>12}{6:>15}".format('Scale(n)', 'time(ms)', 'feval', 'min(x)', 'max(x)', 'sum(x)', 'x(0) + x(1)'))

for n in range(200, 3200, 200):
    elapsed, result = time_function(clp_lp, n)
    s = result[2]
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12f}{4:>12f}{5:>12f}{6:>15}".format(n, elapsed*1000, result[1], s.min(), s.max(), s.sum(), s[0] + s[1]))

Scale(n)    time(ms)       feval      min(x)      max(x)      sum(x)    x(0) + x(1)
200             2.99       -0.82    0.000000    0.010000    1.0000000.015000000005429394
400             2.99       -1.28    0.000000    0.010000    1.0000000.015000000000751215
600             4.00       -1.54    0.000000    0.010000    1.0000000.01500000000851949
800             5.00       -1.63    0.000000    0.010000    1.0000000.015000000002481837
1000            7.00       -1.72    0.000000    0.010000    1.0000000.015000000001100414
1200            6.00       -1.81    0.000000    0.010000    1.0000000.01500000000548405
1400            9.01       -1.90    0.000000    0.010000    1.0000000.015000000001956426
1600           10.00       -1.96    0.000000    0.010000    1.0000000.015000000000082848
1800           10.16       -2.03    0.000000    0.010000    1.0000000.01500000000204834
2000           12.05       -2.06    0.000000    0.010000    1.0000000.0150000000008303
2200           12.99       -2.0